# Do Palace Merging

In [4]:
import pickle
from collections import deque
from pathlib import Path

import graphviz as gv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from gtsam import Pose3
from typing import Tuple, List, Dict, Any, Optional, Set

from gtsfm.graph_partitioner.metis_partitioner import MetisPartitioner
from gtsfm.products.visibility_graph import (
    VisibilityGraph,
    visibility_graph_keys,
)
from gtsfm.utils.io import load_poses, save_poses
from gtsfm.products.cluster_tree import ClusterTree

PALACE = Path("../tests/data/palace")

In [5]:
df = pd.read_csv(PALACE / 'visibility_graph.csv')
graph : VisibilityGraph = list(zip(df["i"], df["j"]))

In [6]:
print("Number of edges in visibility graph:", len(graph))
print("Number of keys:", len(visibility_graph_keys(graph)))

Number of edges in visibility graph: 4139
Number of keys: 281


In [7]:
partitioner = MetisPartitioner()
cluster_tree = partitioner.run(graph)

In [ ]:
with open(PALACE / 'cluster_tree.pkl', 'rb') as f:
    loaded_cluster_tree = pickle.load(f)
print('Loaded cluster_tree from', PALACE / 'cluster_tree.pkl')

In [8]:
cluster_tree

Cluster: [65, 66, 67, 68, 144, 145, 248, 249, 277, 279, 280]
  Cluster: [204, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 279]
    Leaf: [202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239]
    Leaf: [248, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280]
  Cluster: [147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 206]
    Leaf: [183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 19